# Install Packages

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Flatten, Activation, GlobalAvgPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
import itertools
import shutil
import os
import random
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import datetime
from time import time
from my_mobilenet import *

# Using GPU

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Global Variable

In [3]:
TRAIN_PATH = './self_learn_data/seg_train/seg_train/'
VALID_PATH = './self_learn_data/seg_test/seg_test/'
TEST_PATH = './self_learn_data/seg_pred/'
IMAGE_SIZE = (224, 224, 3)
BATCH_SIZES = 64
EPOCHS = 80

# Data Preparetion

In [4]:
datagen = ImageDataGenerator()
train_batches = datagen.flow_from_directory(directory=TRAIN_PATH, color_mode="rgb",target_size=IMAGE_SIZE[:-1], class_mode="categorical", shuffle=True, batch_size=BATCH_SIZES)
valid_batches = datagen.flow_from_directory(directory=VALID_PATH, color_mode="rgb",target_size=IMAGE_SIZE[:-1], class_mode="categorical", shuffle=False, batch_size=BATCH_SIZES)
test_batches = datagen.flow_from_directory(directory=TEST_PATH,target_size=IMAGE_SIZE[:-1],color_mode="rgb", batch_size=BATCH_SIZES, class_mode=None, shuffle=False)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 7301 images belonging to 1 classes.


# Model Construct

In [5]:
md = keras.applications.MobileNet(input_shape = IMAGE_SIZE, weights = None, include_top = False)
x = GlobalAvgPool2D(name='global_avg')(md.output)
prediction = Dense(6, activation='softmax')(x)
model = Model(inputs=md.input, outputs=prediction)
model.summary()
model.load_weights('./model/mobile_imagenetweights.h5')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

# Label unlabeled data

In [6]:
predictions = model.predict(test_batches, verbose=1)

115/115 [==============================] - 9s 63ms/step


In [7]:
image_dict = {0: "buildings", 1: "forest", 2: "glacier", 3: "mountain", 4: "sea", 5: "street"}

In [8]:
from PIL import Image
from tqdm import tqdm
for data_ind, data in tqdm(enumerate(predictions)):
    for ind, softmax in enumerate(data):
        if softmax >= 0.7:   #Expand training data by softmax >= 0.7
            ind1 = data_ind // BATCH_SIZES
            ind2 = data_ind % BATCH_SIZES
            label = image_dict[ind]
            data = Image.fromarray(test_batches[ind1][ind2].astype('uint8'))
            data.save('./self_learn_data/seg_train/seg_train/'+label+'/'+str(len(train_batches)+data_ind)+'.jpg')

7301it [05:18, 22.89it/s]


# Load again

In [9]:
TRAIN_PATH = './self_learn_data/seg_train/seg_train/'
train_batches = datagen.flow_from_directory(directory=TRAIN_PATH, color_mode="rgb",target_size=IMAGE_SIZE[:-1], class_mode="categorical", shuffle=True, batch_size=BATCH_SIZES)

Found 20243 images belonging to 6 classes.


In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [11]:
freeze_layer = -1
for ind, layer in enumerate(model.layers):
    if layer.name == "conv_dw_10":
        freeze_layer = ind
        print("Freeze layer: ", freeze_layer)
for layer in model.layers[:freeze_layer]:
    layer.trainable = False
for layer in model.layers[freeze_layer:]:
    layer.trainable = True

Freeze layer:  61


In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

# FineTune from old model (lr = 1e-5)

In [13]:
optimizer = Adam(learning_rate = 1e-5)
model.compile(
    loss='categorical_crossentropy',
    optimizer= optimizer,
    metrics=['accuracy']
)
stop_callback = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience = 4, verbose=1, min_lr = 1e-6)
t1 = time()
history = model.fit(train_batches, validation_data = valid_batches, epochs= EPOCHS, steps_per_epoch=len(train_batches), validation_steps=len(valid_batches), callbacks = [stop_callback,reduce_lr], shuffle = True)
t2 = time()
print("execution time: ", t2 - t1)

Epoch 1/80
317/317 [==============================] - 26s 78ms/step - loss: 0.0133 - accuracy: 0.9962 - val_loss: 0.2841 - val_accuracy: 0.9300
Epoch 2/80
317/317 [==============================] - 24s 75ms/step - loss: 0.0125 - accuracy: 0.9959 - val_loss: 0.2878 - val_accuracy: 0.9307
Epoch 3/80
317/317 [==============================] - 24s 75ms/step - loss: 0.0104 - accuracy: 0.9973 - val_loss: 0.2934 - val_accuracy: 0.9307
Epoch 4/80
317/317 [==============================] - 24s 75ms/step - loss: 0.0096 - accuracy: 0.9972 - val_loss: 0.2988 - val_accuracy: 0.9307
Epoch 5/80
317/317 [==============================] - 24s 75ms/step - loss: 0.0090 - accuracy: 0.9972 - val_loss: 0.3028 - val_accuracy: 0.9297

Epoch 00005: ReduceLROnPlateau reducing learning rate to 1e-06.
Epoch 6/80
317/317 [==============================] - 24s 75ms/step - loss: 0.0100 - accuracy: 0.9974 - val_loss: 0.3027 - val_accuracy: 0.9293
Epoch 7/80
317/317 [==============================] - 24s 75ms/step - l

In [14]:
scoreSeg = model.evaluate(valid_batches)
print("Test Data Accuracy = ",scoreSeg[1])

47/47 [==============================] - 3s 63ms/step - loss: 0.2878 - accuracy: 0.9307
Test Data Accuracy =  0.9306666851043701


# Retrain with Imagenet Weight (lr = 1e-3)

In [15]:
md = keras.applications.MobileNet(input_shape = IMAGE_SIZE, weights = "imagenet", include_top = False)
x = GlobalAvgPool2D(name='global_avg')(md.output)
prediction = Dense(6, activation='softmax')(x)
model = Model(inputs=md.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0   

In [16]:
optimizer = Adam(learning_rate = 1e-3)
model.compile(
    loss='categorical_crossentropy',
    optimizer= optimizer,
    metrics=['accuracy']
)
stop_callback = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience = 4, verbose=1, min_lr = 1e-6)
t1 = time()
history = model.fit(train_batches, validation_data = valid_batches, epochs= EPOCHS, steps_per_epoch=len(train_batches), validation_steps=len(valid_batches), callbacks = [stop_callback,reduce_lr], shuffle = True)
t2 = time()
print("execution time: ", t2 - t1)

Epoch 1/80
317/317 [==============================] - 71s 219ms/step - loss: 0.4344 - accuracy: 0.8652 - val_loss: 0.4761 - val_accuracy: 0.8667
Epoch 2/80
317/317 [==============================] - 69s 218ms/step - loss: 0.1612 - accuracy: 0.9469 - val_loss: 0.5064 - val_accuracy: 0.8690
Epoch 3/80
317/317 [==============================] - 69s 218ms/step - loss: 0.1340 - accuracy: 0.9543 - val_loss: 0.4608 - val_accuracy: 0.8543
Epoch 4/80
317/317 [==============================] - 68s 216ms/step - loss: 0.1004 - accuracy: 0.9653 - val_loss: 0.3581 - val_accuracy: 0.8907
Epoch 5/80
317/317 [==============================] - 69s 217ms/step - loss: 0.0996 - accuracy: 0.9652 - val_loss: 0.4362 - val_accuracy: 0.8687
Epoch 6/80
317/317 [==============================] - 69s 218ms/step - loss: 0.0673 - accuracy: 0.9766 - val_loss: 0.4293 - val_accuracy: 0.8987
Epoch 7/80
317/317 [==============================] - 69s 217ms/step - loss: 0.0631 - accuracy: 0.9777 - val_loss: 0.3827 - val_ac

In [17]:
scoreSeg = model.evaluate(valid_batches)
print("Test Data Accuracy = ",scoreSeg[1])

47/47 [==============================] - 3s 63ms/step - loss: 0.3206 - accuracy: 0.9343
Test Data Accuracy =  0.934333324432373


In [19]:
# predict classes
from sklearn.metrics import classification_report
test_labels = valid_batches.classes 
predictions = model.predict_generator(valid_batches, verbose=1)
y_pred = np.argmax(predictions, axis=-1)
print(classification_report(test_labels, y_pred, target_names = valid_batches.class_indices))

47/47 [==============================] - ETA:  - 3s 63ms/step
              precision    recall  f1-score   support

   buildings       0.93      0.93      0.93       437
      forest       0.99      0.99      0.99       474
     glacier       0.91      0.88      0.90       553
    mountain       0.90      0.90      0.90       525
         sea       0.94      0.97      0.96       510
      street       0.94      0.94      0.94       501

    accuracy                           0.93      3000
   macro avg       0.94      0.94      0.94      3000
weighted avg       0.93      0.93      0.93      3000

